# Лабораторная работа №3. Реализация сверточной нейронной сети

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import cv2

train_dir = "notMNIST_small/train"
test_dir = "notMNIST_small/test"

2025-05-01 01:33:10.308485: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-01 01:33:10.309156: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-01 01:33:10.312233: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-01 01:33:10.321311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746052390.336659    4008 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746052390.34

Функция загрузки данных из папок

In [2]:
def load_data(folder):
    images = []
    labels = []
    label_map = {chr(i): idx for idx, i in enumerate(range(ord('A'), ord('J')+1))}

    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for file in os.listdir(label_path):
                img_path = os.path.join(label_path, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Ошибка загрузки изображения: {img_path}")
                    continue  # Пропускаем этот файл
                img = cv2.resize(img, (28, 28))
                images.append(img)
                labels.append(label_map[label])

    return np.array(images), np.array(labels)

Загрузка и нормализация данных

In [3]:
train_images, train_labels = load_data(train_dir)
test_images, test_labels = load_data(test_dir)

train_images = train_images / 255.0
test_images = test_images / 255.0

Ошибка загрузки изображения: notMNIST_small/train/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png
Ошибка загрузки изображения: notMNIST_small/train/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png


Преобразование формы данных и определние модели

In [4]:
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

**Задание 1.** Реализуйте нейронную сеть с двумя сверточными слоями, и одним полносвязным с нейронами с кусочно-линейной функцией активации. Какова точность построенное модели?

Предлагается создать модель, реализующую несколько слоев: 
- `Сверточные слои` - хороши при проведении локального анализа различий
- `Полносвязный слой` - необходим для контроля глобального контекста и принятия решений

Проектирование модели и инициализация обучения

In [5]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), # Сверточный
    layers.Conv2D(64, (3, 3), activation='relu'), # Сверточный
    layers.Flatten(), # Преобразование многомерного тензора в одномерный массив
    layers.Dense(128, activation='relu'), # Полносвязный
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))


/home/relsa/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/envs/env_1/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-01 01:33:14.803406: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.8185 - loss: 0.6293 - val_accuracy: 0.9227 - val_loss: 0.2559
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9493 - loss: 0.1676 - val_accuracy: 0.9289 - val_loss: 0.2289
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9715 - loss: 0.0942 - val_accuracy: 0.9313 - val_loss: 0.2620
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.9836 - loss: 0.0548 - val_accuracy: 0.9281 - val_loss: 0.3036
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9893 - loss: 0.0346 - val_accuracy: 0.9313 - val_loss: 0.3260
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9908 - loss: 0.0351 - val_accuracy: 0.9321 - val_loss: 0.3775
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9932 - loss: 0.0261 - val_accuracy: 0.9281 - val_loss: 0.3904
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9952 - loss: 0.0188 - 

Оценка точности

In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Точность модели: {test_acc * 100:.2f}%\nПотери: {test_loss}')

117/117 - 1s - 12ms/step - accuracy: 0.9294 - loss: 0.4257
Точность модели: 92.94%
Потери: 0.42569679021835327


Обучение шло довольно медленно. Точность модели на обучающем массиве данных была крайне высокая (`99%`) при этом на тестовом наборе она резко упала до `93%`, что свидетельствует о переобучении.

**Задание 2.** Замените сверточные слои на слои, реализующие операцию пулинга (Pooling) с функцией максимума или среднего. Как это повлияло на точность классификатора?

`Пуллинг` позволит значительно сократить вычислительную сложность обучения модели и снизить риск переобучения.

Проектирование модели и инициализация обучения

In [7]:
model = keras.Sequential([
    layers.MaxPooling2D((2, 2), input_shape=(28, 28, 1)),
    layers.AveragePooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

/home/relsa/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/envs/env_1/lib/python3.12/site-packages/keras/src/layers/pooling/base_pooling.py:23: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(name=name, **kwargs)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6504 - loss: 1.2829 - val_accuracy: 0.8851 - val_loss: 0.4633
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8734 - loss: 0.4765 - val_accuracy: 0.8897 - val_loss: 0.4186
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8893 - loss: 0.4147 - val_accuracy: 0.8977 - val_loss: 0.3824
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8916 - loss: 0.3938 - val_accuracy: 0.8988 - val_loss: 0.3766
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8958 - loss: 0.3792 - val_accuracy: 0.9053 - val_loss: 0.3492
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9004 - loss: 0.3550 - val_accuracy: 0.9071 - val_loss: 0.3356
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9048 - loss: 0.3346 - val_accuracy: 0.9074 - val_loss: 0.3364
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9121 - loss: 0.3176 - val_accuracy: 0.

Оценка точности

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Точность модели: {test_acc * 100:.2f}%\nПотери: {test_loss}')

117/117 - 0s - 2ms/step - accuracy: 0.9138 - loss: 0.3089
Точность модели: 91.38%
Потери: 0.3089263439178467


Видно, что обучение прошло крайне быстро. При этом результаты на обучающей выборке примерно равны результатам на тестовой. Однако сам по себе результат в `91%` является регрессом по сравнению с моделью, построенной на сверточных слоях. 

Это объяснятся тем, что при `пуллинге` уменьшается размер признакового пространства, сокращая количество параметров и вычислительных затрат, однако падает и чувствительность модели к мелким, но важным деталям, что сильно влияет на точность.

**Задание 3.** Реализуйте классическую архитектуру сверточных сетей [LeNet-5](http://yann.lecun.com/exdb/lenet/). 

Архитектура `LeNet-5` является отличным компромиссом, который соединяет в себе точность `сверточных` слоев и оптимизацию, которую дает использование `пуллинга`. 

Общая схема архитектуры выглядит следующим образом: 

![alt](https://upload.wikimedia.org/wikipedia/commons/5/5e/LeNet-5_architecture_block_diagram.svg)

Как видно, имеется две пары слоев {`сверточный`, `пуллинг`} и три полносвязных слоя, один из которых является выходным.


Проектирование модели и инициализация обучения

In [9]:
model = keras.Sequential([
    layers.Conv2D(6, (5, 5), activation='tanh', padding='same', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(16, (5, 5), activation='tanh'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(120, activation='tanh'),
    layers.Dense(84, activation='tanh'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7536 - loss: 0.8231 - val_accuracy: 0.8975 - val_loss: 0.3543
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9068 - loss: 0.3148 - val_accuracy: 0.9192 - val_loss: 0.2767
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9238 - loss: 0.2506 - val_accuracy: 0.9294 - val_loss: 0.2496
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9403 - loss: 0.1899 - val_accuracy: 0.9257 - val_loss: 0.2501
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9537 - loss: 0.1525 - val_accuracy: 0.9308 - val_loss: 0.2329
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9602 - loss: 0.1257 - val_accuracy: 0.9353 - val_loss: 0.2271
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9658 - loss: 0.1077 - val_accuracy: 0.9361 - val_loss: 0.2216
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9760 - loss: 0.0812 - val_accuracy: 0.

Оценка точности

In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Точность модели LeNet-5: {test_acc * 100:.2f}%\nПотери: {test_loss}')

117/117 - 0s - 2ms/step - accuracy: 0.9348 - loss: 0.2478
Точность модели LeNet-5: 93.48%
Потери: 0.24784143269062042


Использование `пуллинга` совместно со `сверточными` слоями действительно помогло снизить влияние переобучения и оптимизировало сам обучающий процесс. 

В итоге точность модели в итоге повысилась до `94%`, при этом время обучения сократилось на `56 секунд`, а разница между точностью на обучающей и тестовой выборках уменьшилась на `1,86%`